In [8]:
import numpy as np
import open3d as o3d
from tqdm import tqdm

In [9]:
datas = o3d.io.read_point_cloud('Line11.ply')
xyz = np.asarray(datas.points)

In [10]:
def estimate_parameters(pts):
        # 最小二乘法估算平面模型
        # 只有三个点时，可以直接计算

        c = np.mean(pts,axis=0)
        l1 = pts[1]-pts[0]
        l2 = pts[2]-pts[0]
        n = np.cross(l1,l2)
        scale = [n[i]**2 for i in range(n.shape[0])]
        #print(scale)
        n = n/np.sqrt(np.sum(scale))

        params = np.hstack((c.reshape(1,-1),n.reshape(1,-1)))[0,:]
        return params

In [15]:
xyz = xyz[np.lexsort(xyz.T)]
Nlrp = 1000
LRP = np.average(xyz[:Nlrp,2])
Thseeds = 0.1
seed = xyz[xyz[:,2] < Thseeds + LRP ]
seed_cp = seed
for i in range(10):    
    params = estimate_parameters(pts=seed_cp)
    h = (xyz[:,0]-params[0]) * params[3] + (xyz[:,1]-params[1])* params[4]+((xyz[:,2]-params[2])* params[5])/(np.sqrt(params[3]**2+params[4]**2+params[5]**2))
    seed_cp = xyz[h < 0.1]

In [12]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(seed_cp)
o3d.visualization.draw_geometries([pcd])
pcd.estimate_normals()

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 


In [6]:
np.asarray(pcd.normals)

array([[ 0.78806726, -0.41248379,  0.45695417],
       [ 0.78960853, -0.40809106,  0.45823582],
       [-0.04916668, -0.05902361,  0.99704506],
       ...,
       [-0.62502192,  0.75291568,  0.20607178],
       [-0.56807341,  0.74923855,  0.34049111],
       [-0.54862958,  0.80643911,  0.22059361]])

In [7]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(seed_cp)
o3d.visualization.draw_geometries([pcd])